In [188]:
from transformers import CLIPProcessor
import random
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import matplotlib.pyplot as plt
import numpy as np
import tarfile
import os
from PIL import Image

from timeit import default_timer as timer
from tqdm.auto import tqdm
from torchvision import datasets, models, transforms
from torchsummary import summary
from typing import Callable, Dict, List, Tuple, Union
from torch.optim import lr_scheduler

import clip #importing clip model

In [189]:
device = "cpu"
if torch.cuda.is_available:
  print('cuda available')
  device = torch.device("cuda:0")
else:
  print('Please set GPU via Edit -> Notebook Settings.')

cuda available


In [190]:
# CLIP 모델과 tokenizer를 로드합니다.
model, preprocess = clip.load('ViT-B/32', device=device)
tokenizer = clip.tokenize

In [191]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=False) #exploiting CLIP preprocess for dataset transformation

In [192]:
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

In [193]:
# 데이터셋 로드
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=False) #exploiting CLIP preprocess for dataset transformation

In [194]:
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(cifar100, [40000, 8000,2000])
# print(cifar100)

In [195]:
# 이미지와 텍스트를 가져와서 임베딩을 생성합니다.
image_input = torch.tensor(np.stack([test_set[x][0] for x in range(len(test_set))])).to(device)

# test_set에 있는 이미지들의 클래스 정보를 가져옵니다.
test_classes = [cifar100.classes[idx] for (_, idx) in test_set]

# 클래스 정보를 이용하여 텍스트를 생성합니다.
text_input = [f"a photo of {label}" for label in test_classes]

# 텍스트를 토큰화하여 텐서로 변환합니다.
text_input = clip.tokenize(text_input).to(device)

with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_input).float()

# 이미지와 텍스트 임베딩을 결합합니다.
features = torch.cat([image_features, text_features], dim = -1)

In [201]:
features[0]

tensor([-0.0510,  0.0335, -0.1149,  ...,  0.0068,  0.0726,  0.4988],
       device='cuda:0')

In [198]:
print(features.shape)
print(image_features.shape)
print(image_features.shape)

torch.Size([2000, 1024])
torch.Size([2000, 512])
torch.Size([2000, 512])


In [178]:
print(f'Num of data : len(test_classes)')
print(f'Num of class : len(set(test_classes))')

Num of data : len(test_classes)
Num of class : len(set(test_classes))


In [182]:
num_workers = 4
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size = batch_size, #배치사이즈
                                               shuffle = True,
                                               num_workers=num_workers
                                               )

val_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size = batch_size,                                              
                                               shuffle = True,
                                                num_workers=num_workers
                                              )